In [27]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from pycaret.regression import setup, compare_models
import logging
import transformers
from sklearn.metrics import accuracy_score
import optuna
from sklearn.model_selection import KFold
from sklearn.experimental import enable_iterative_imputer

trainUsers=pd.read_csv("users_train.csv")
trainUsers.drop("ID",axis=1,inplace=True)
testUsers=pd.read_csv("users_test.csv")
testUsers.drop("ID",axis=1,inplace=True)
trainFeatures=pd.read_csv("user_features_train.csv")
testFeatures=pd.read_csv("user_features_test.csv")
target=pd.read_csv("targets_train.csv")
train=[trainUsers,trainFeatures,target]
test=[testUsers,testFeatures]
train=pd.concat(train,axis=1)
test=pd.concat(test,axis=1)


ID=test["ID"]

In [28]:
def process_datetime_features(df):
    # `first_open_date` ve `first_open_timestamp` sütunlarının tarih formatında olduğundan emin olun
    df['first_open_date'] = pd.to_datetime(df['first_open_date'])
    df['first_open_timestamp'] = pd.to_datetime(df['first_open_timestamp'])
    df['local_first_open_timestamp'] = pd.to_datetime(df['local_first_open_timestamp'])

    # Tarih bileşenlerini ayırın
    df['open_year'] = df['first_open_date'].dt.year
    df['open_month'] = df['first_open_date'].dt.month
    df['open_day'] = df['first_open_date'].dt.day

    # `microsecond` değerlerine erişim
    df["micro_first_open_times"] = df['first_open_timestamp'].dt.microsecond
    df["local_micro_first_open_times"] = df['local_first_open_timestamp'].dt.microsecond

    # Orijinal datetime sütunlarını kaldırın
    df = df.drop(columns=['first_open_date', 'first_open_timestamp', 'local_first_open_timestamp'],inplace=True)

    return df
process_datetime_features(train)
process_datetime_features(test)
process_datetime_features(testUsers)

In [29]:
trainUserscopy=trainUsers.iloc[:10000]
testUserscopy=testUsers.iloc[:10000]
trainFeaturescopy=trainFeatures.iloc[:10000]
testFeaturescopy=testFeatures.iloc[:10000]


In [30]:
train.shape,test.shape

((878594, 90), (585730, 88))

In [31]:
train.drop("ID",inplace=True,axis=1)
test.drop("ID",inplace=True,axis=1)

test.shape,train.shape


((585730, 87), (878594, 88))

In [32]:
#çalıştır
def check_data(dataframe):
 print(20*"-" + "Information".center(20) + 20*"-")
 print(dataframe.info())
 print("\n" + 20*"-" + "The First 5 Data".center(20) + 20*"-")
 print(dataframe.head(20))
 print(20*"-" + "Data Shape".center(20) + 20*"-")
 print(dataframe.shape)

check_data(train)



--------------------    Information     --------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878594 entries, 0 to 878593
Data columns (total 88 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   country                       878512 non-null  object 
 1   platform                      878594 non-null  object 
 2   device_category               878594 non-null  object 
 3   device_brand                  872754 non-null  object 
 4   device_model                  878594 non-null  object 
 5   has_ios_att_permission        878594 non-null  bool   
 6   ad_network                    568124 non-null  object 
 7   first_prediction              852859 non-null  float64
 8   RetentionD0                   878594 non-null  bool   
 9   RetentionD1                   878594 non-null  bool   
 10  RetentionD2                   878594 non-null  bool   
 11  RetentionD3                   878594 non-nu

In [33]:
def grab_col_names(dataframe, cat_th, car_th):
    """
    Grabs the column names according to data type and unique value counts.

    Parameters:
    - dataframe: pandas DataFrame to evaluate.
    - cat_th: Threshold for the number of unique values to consider a numerical column as categorical.
    - car_th: Threshold for the number of unique values to consider a categorical column as cardinal.

    Returns:
    - cat_cols: List of categorical column names.
    - cat_but_car: List of categorical but cardinal column names.
    - num_but_cat: List of numerical columns considered as categorical due to low number of unique values.
    - num_cols: List of numerical column names.
    """


    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]
    # Categorical columns that are not considered cardinal
    cat_but_not_car = [col for col in cat_cols if col not in cat_but_car]
    

# Display unique value counts for all columns
    uniquevalue = dataframe.nunique().sort_values(ascending=False)
    print(f"The unique values:\n{uniquevalue}")
    
    # Output column lists
    print(f"cat_cols: {cat_cols}")
    print(f'num_cols: {num_cols}')
    print(f'num_cols but cat: {num_but_cat}')
    print(f'cat_but_car: {cat_but_car}')
    print(f'cat_but_not_car: {cat_but_not_car}')
    
    # Return the column lists
    return cat_cols, cat_but_car, num_but_cat, num_cols, cat_but_not_car

# Test the function with both train and test datasets

print("\n")
cat_cols_testFeatures, cat_but_car_testFeatures, num_but_cat_testFeatures, num_cols_testFeatures, cat_but_not_car_testFeatures = grab_col_names(testFeaturescopy, 10, 30)
print("\n")

cat_cols_testUsers, cat_but_car_testUsers, num_but_cat_testUsers, num_cols_testUsers, cat_but_not_car_testUsers = grab_col_names(testUserscopy, 10, 30)






The unique values:
ID                  10000
first_prediction     7751
AdRevenueD0          4240
AdRevenueD1          2941
AdRevenueD2          2205
                    ...  
IAPRevenueD7            1
IAPRevenueD5            1
IAPRevenueD0            1
RetentionD0             1
IAPRevenueD15           1
Length: 76, dtype: int64
cat_cols: ['RetentionD0', 'RetentionD1', 'RetentionD2', 'RetentionD3', 'RetentionD4', 'RetentionD5', 'RetentionD6', 'RetentionD7', 'RetentionD8', 'RetentionD9', 'RetentionD10', 'RetentionD11', 'RetentionD12', 'RetentionD13', 'RetentionD14', 'RetentionD15', 'IAPRevenueD0', 'IAPRevenueD1', 'IAPRevenueD2', 'IAPRevenueD3', 'IAPRevenueD4', 'IAPRevenueD5', 'IAPRevenueD6', 'IAPRevenueD7', 'IAPRevenueD8', 'IAPRevenueD9', 'IAPRevenueD10', 'IAPRevenueD11', 'IAPRevenueD12', 'IAPRevenueD13', 'IAPRevenueD14', 'IAPRevenueD15']
num_cols: ['ID', 'first_prediction', 'LevelAdvancedCountD0', 'LevelAdvancedCountD1', 'LevelAdvancedCountD2', 'LevelAdvancedCountD3', 'LevelAdvancedCo

In [34]:
#çalıştır
def checknull(data,percentvalue):
    
    upperPercentValueColumnlist=[col for col in data.columns if data[col].isnull().mean()*100>percentvalue]
    #boş değeri %15'ten fazla olan sütunlar
    betweenPercentValueColumnlist=[col for col in data.columns if data[col].isnull().mean()*100<percentvalue and data[col].isnull().mean()*100>0]
    #boş değeri %0-15 arasında olan sütunlar
    zeroPercentnullValueColumnlist=[col for col in data.columns if data[col].isnull().mean()==0]
    # boş değeri 0 olan değerler
    null_percent=(data.isnull().mean()*100).sort_values(ascending=False)
    
    print(f"""
The null values percent data:

    {null_percent}
          
          
Columns with null values greater than {percentvalue}%: {upperPercentValueColumnlist} 

Columns with null values is 0% : {zeroPercentnullValueColumnlist}

Columns with null values between 0 - {percentvalue}%: {betweenPercentValueColumnlist} 

""")
    return upperPercentValueColumnlist,betweenPercentValueColumnlist,zeroPercentnullValueColumnlist,null_percent

upperPercentValueColumnlistTrainFeatures,betweenPercentValueColumnlistTrainFeatures,zeroPercentnullValueColumnlistTrainFeatures,null_percentTrainFeatures=checknull(trainFeaturescopy,45)
upperPercentValueColumnlistTestFeatures,betweenPercentValueColumnlistTestFeatures,zeroPercentnullValueColumnlistTestFeatures,null_percentTestFeatures=checknull(test,45)
upperPercentValueColumnlistTestUsers,betweenPercentValueColumnlistTestUsers,zeroPercentnullValueColumnlistTestUsers,null_percentTestUsers=checknull(testUserscopy,45)
upperPercentValueColumnlistTrainUsers,betweenPercentValueColumnlistTrainUsers,zeroPercentnullValueColumnlistTrainUsers,null_percentTrainUsers=checknull(trainUserscopy,45)






The null values percent data:

    Level_10_Duration        40.42
Level_9_Duration         36.32
Level_8_Duration         34.35
Level_7_Duration         30.92
Level_6_Duration         26.24
                         ...  
LevelAdvancedCountD10     0.00
LevelAdvancedCountD11     0.00
LevelAdvancedCountD12     0.00
LevelAdvancedCountD13     0.00
IAPRevenueD15             0.00
Length: 76, dtype: float64
          
          
Columns with null values greater than 45%: [] 

Columns with null values is 0% : ['ID', 'RetentionD0', 'RetentionD1', 'RetentionD2', 'RetentionD3', 'RetentionD4', 'RetentionD5', 'RetentionD6', 'RetentionD7', 'RetentionD8', 'RetentionD9', 'RetentionD10', 'RetentionD11', 'RetentionD12', 'RetentionD13', 'RetentionD14', 'RetentionD15', 'LevelAdvancedCountD0', 'LevelAdvancedCountD1', 'LevelAdvancedCountD2', 'LevelAdvancedCountD3', 'LevelAdvancedCountD4', 'LevelAdvancedCountD5', 'LevelAdvancedCountD6', 'LevelAdvancedCountD7', 'LevelAdvancedCountD8', 'LevelAdvancedCountD9', 

def gra(dataframe, cat_th, car_th):
    """
    Parameters (for null values):
    dataframe (pd.DataFrame): DataFrame to analyze.
    cat_th (int): Threshold for numeric columns to be treated as categorical.
    car_th (int): Threshold for categorical columns to be considered cardinal.
    """

    # Columns with null values that are categorical
    null_value_cat_cols = [col for col in zeroPercentnullValueColumnlistTestFeatures if dataframe[col].dtypes == "O"]

    # Numeric columns with null values but few unique values (treated as categorical)
    null_value_num_but_cat = [
        col for col in zeroPercentnullValueColumnlistTestFeatures if dataframe[col].nunique() < cat_th and dataframe[col].dtypes != "O"]

    # Combine categorical and numeric-as-categorical columns
    null_value_cat_cols += null_value_num_but_cat

    # Categorical columns with many unique values (cardinal)
    null_value_cat_but_car = [
        col for col in zeroPercentnullValueColumnlistTestFeatures if dataframe[col].nunique() > car_th and dataframe[col].dtypes == "O"]

    null_num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    
    null_num_cols = [col for col in null_num_cols if col not in null_value_cat_cols]

    # Print number of unique values
    print(f"The unique values:\n {dataframe[zeroPercentnullValueColumnlistTestFeatures].nunique().sort_values(ascending=False)}")

    # Print categorized columns
    print(f"Categorical columns with nulls: {null_value_cat_cols}")
    print(f'Cardinal categorical columns with nulls: {null_value_cat_but_car}')
    print(f'Numeric columns with nulls (treated as categorical): {null_value_num_but_cat}')

    return null_value_cat_cols, null_value_cat_but_car, null_value_num_but_cat, null_num_cols

# Example usage
null_value_cat_cols_user, null_value_num_but_cat_user, null_value_cat_but_car_user,null_num_cols = grab_col_names(test, 10, 20)




# IAPRevenue için benzer sütunlar oluşturma
train["IAPRevenueTotal2"] = train["IAPRevenueD0"] + train["IAPRevenueD1"]
train["IAPRevenueTotal4"] = train["IAPRevenueD2"] + train["IAPRevenueD3"]
train["IAPRevenueTotal6"] = train["IAPRevenueD4"] + train["IAPRevenueD5"]
train["IAPRevenueTotal8"] = train["IAPRevenueD6"] + train["IAPRevenueD7"]
train["IAPRevenueTotal10"] = train["IAPRevenueD8"] + train["IAPRevenueD9"]
train["IAPRevenueTotal12"] = train["IAPRevenueD10"] + train["IAPRevenueD11"]
train["IAPRevenueTotal14"] = train["IAPRevenueD12"] + train["IAPRevenueD13"]
train["IAPRevenueTotal16"] = train["IAPRevenueD14"] + train["IAPRevenueD15"]


# LevelAdvancedCountTotal
train["LevelAdvancedCountTotal2"] = train["LevelAdvancedCountD0"] + train["LevelAdvancedCountD1"]
train["LevelAdvancedCountTotal4"] = train["LevelAdvancedCountD2"] + train["LevelAdvancedCountD3"]
train["LevelAdvancedCountTotal6"] = train["LevelAdvancedCountD4"] + train["LevelAdvancedCountD5"]
train["LevelAdvancedCountTotal8"] = train["LevelAdvancedCountD6"] + train["LevelAdvancedCountD7"]
train["LevelAdvancedCountTotal10"] = train["LevelAdvancedCountD8"] + train["LevelAdvancedCountD9"]
train["LevelAdvancedCountTotal12"] = train["LevelAdvancedCountD10"] + train["LevelAdvancedCountD11"]
train["LevelAdvancedCountTotal14"] = train["LevelAdvancedCountD12"] + train["LevelAdvancedCountD13"]
train["LevelAdvancedCountTotal16"] = train["LevelAdvancedCountD14"] + train["LevelAdvancedCountD15"]

# Level Duration
train["LevelDurationTotal2"] = train["Level_1_Duration"] + train["Level_2_Duration"]
train["LevelDurationTotal4"] = train["Level_3_Duration"] + train["Level_4_Duration"]
train["LevelDurationTotal6"] = train["Level_5_Duration"] + train["Level_6_Duration"]
train["LevelDurationTotal8"] = train["Level_7_Duration"] + train["Level_8_Duration"]
train["LevelDurationTotal10"] = train["Level_9_Duration"] + train["Level_10_Duration"]

# AdRevenue
train["AdRevenueTotal2"] = train["AdRevenueD0"] + train["AdRevenueD1"]
train["AdRevenueTotal4"] = train["AdRevenueD2"] + train["AdRevenueD3"]
train["AdRevenueTotal6"] = train["AdRevenueD4"] + train["AdRevenueD5"]
train["AdRevenueTotal8"] = train["AdRevenueD6"] + train["AdRevenueD7"]
train["AdRevenueTotal10"] = train["AdRevenueD8"] + train["AdRevenueD9"]
train["AdRevenueTotal12"] = train["AdRevenueD10"] + train["AdRevenueD11"]
train["AdRevenueTotal14"] = train["AdRevenueD12"] + train["AdRevenueD13"]
train["AdRevenueTotal16"] = train["AdRevenueD14"] + train["AdRevenueD15"]


# Retention için benzer sütunlar oluşturma (train seti)
train["RetentionTotal2"] = train["RetentionD0"] + train["RetentionD1"]
train["RetentionTotal4"] = train["RetentionD2"] + train["RetentionD3"]
train["RetentionTotal6"] = train["RetentionD4"] + train["RetentionD5"]
train["RetentionTotal8"] = train["RetentionD6"] + train["RetentionD7"]
train["RetentionTotal10"] = train["RetentionD8"] + train["RetentionD9"]
train["RetentionTotal12"] = train["RetentionD10"] + train["RetentionD11"]
train["RetentionTotal14"] = train["RetentionD12"] + train["RetentionD13"]
train["RetentionTotal16"] = train["RetentionD14"] + train["RetentionD15"]



# LevelAdvancedCountTotal
test["LevelAdvancedCountTotal2"] = test["LevelAdvancedCountD0"] + test["LevelAdvancedCountD1"]
test["LevelAdvancedCountTotal4"] = test["LevelAdvancedCountD2"] + test["LevelAdvancedCountD3"]
test["LevelAdvancedCountTotal6"] = test["LevelAdvancedCountD4"] + test["LevelAdvancedCountD5"]
test["LevelAdvancedCountTotal8"] = test["LevelAdvancedCountD6"] + test["LevelAdvancedCountD7"]
test["LevelAdvancedCountTotal10"] = test["LevelAdvancedCountD8"] + test["LevelAdvancedCountD9"]
test["LevelAdvancedCountTotal12"] = test["LevelAdvancedCountD10"] + test["LevelAdvancedCountD11"]
test["LevelAdvancedCountTotal14"] = test["LevelAdvancedCountD12"] + test["LevelAdvancedCountD13"]
test["LevelAdvancedCountTotal16"] = test["LevelAdvancedCountD14"] + test["LevelAdvancedCountD15"]

# Level Duration
test["LevelDurationTotal2"] = test["Level_1_Duration"] + test["Level_2_Duration"]
test["LevelDurationTotal4"] = test["Level_3_Duration"] + test["Level_4_Duration"]
test["LevelDurationTotal6"] = test["Level_5_Duration"] + test["Level_6_Duration"]
test["LevelDurationTotal8"] = test["Level_7_Duration"] + test["Level_8_Duration"]
test["LevelDurationTotal10"] = test["Level_9_Duration"] + test["Level_10_Duration"]

# AdRevenue
test["AdRevenueTotal2"] = test["AdRevenueD0"] + test["AdRevenueD1"]
test["AdRevenueTotal4"] = test["AdRevenueD2"] + test["AdRevenueD3"]
test["AdRevenueTotal6"] = test["AdRevenueD4"] + test["AdRevenueD5"]
test["AdRevenueTotal8"] = test["AdRevenueD6"] + test["AdRevenueD7"]
test["AdRevenueTotal10"] = test["AdRevenueD8"] + test["AdRevenueD9"]
test["AdRevenueTotal12"] = test["AdRevenueD10"] + test["AdRevenueD11"]
test["AdRevenueTotal14"] = test["AdRevenueD12"] + test["AdRevenueD13"]
test["AdRevenueTotal16"] = test["AdRevenueD14"] + test["AdRevenueD15"]


# IAPRevenue için benzer sütunlar oluşturma (test seti)
test["IAPRevenueTotal2"] = test["IAPRevenueD0"] + test["IAPRevenueD1"]
test["IAPRevenueTotal4"] = test["IAPRevenueD2"] + test["IAPRevenueD3"]
test["IAPRevenueTotal6"] = test["IAPRevenueD4"] + test["IAPRevenueD5"]
test["IAPRevenueTotal8"] = test["IAPRevenueD6"] + test["IAPRevenueD7"]
test["IAPRevenueTotal10"] = test["IAPRevenueD8"] + test["IAPRevenueD9"]
test["IAPRevenueTotal12"] = test["IAPRevenueD10"] + test["IAPRevenueD11"]
test["IAPRevenueTotal14"] = test["IAPRevenueD12"] + test["IAPRevenueD13"]
test["IAPRevenueTotal16"] = test["IAPRevenueD14"] + test["IAPRevenueD15"]


# Retention için benzer sütunlar oluşturma (test seti)
test["RetentionTotal2"] = test["RetentionD0"] + test["RetentionD1"]
test["RetentionTotal4"] = test["RetentionD2"] + test["RetentionD3"]
test["RetentionTotal6"] = test["RetentionD4"] + test["RetentionD5"]
test["RetentionTotal8"] = test["RetentionD6"] + test["RetentionD7"]
test["RetentionTotal10"] = test["RetentionD8"] + test["RetentionD9"]
test["RetentionTotal12"] = test["RetentionD10"] + test["RetentionD11"]
test["RetentionTotal14"] = test["RetentionD12"] + test["RetentionD13"]
test["RetentionTotal16"] = test["RetentionD14"] + test["RetentionD15"]



#çalıştır

def grab_col_names(dataframe, cat_th, car_th, column_list):
    """
    Grabs the column names according to data type and unique value counts from a specific column list.

    Parameters:
    - dataframe: pandas DataFrame to evaluate.
    - cat_th: Threshold for the number of unique values to consider a numerical column as categorical.
    - car_th: Threshold for the number of unique values to consider a categorical column as cardinal.
    - column_list: List of columns to evaluate instead of all columns.

    Returns:
    - cat_cols: List of categorical column names.
    - cat_but_car: List of categorical but cardinal column names.
    - num_but_cat: List of numerical columns considered as categorical due to low number of unique values.
    """
    
    # Categorical columns within the specific column list
    string_categoric_cols = [col for col in column_list if dataframe[col].dtype == "O" and dataframe[col].nunique() < cat_th]
    
    # Numerical but categorical columns within the specific column list
    numeric_cols = [col for col in column_list if dataframe[col].nunique() > cat_th and dataframe[col].dtype != "O"]
    

    
    # Categorical but cardinal columns within the specific column list
    cardinal_col = [col for col in column_list if dataframe[col].nunique() > car_th and dataframe[col].dtype == "O"]

 
    
    # Print unique value counts for the specified columns
    print(f"The unique values:\n {dataframe[column_list].nunique().sort_values(ascending=False)}")
    
    # Output column lists
    print(f"null column string categoric columns: {string_categoric_cols}")
    print(f'null column numeric: {numeric_cols}')
    print(f'null column cardinal columns: {cardinal_col}')
    

    
    # Return the column lists
    return string_categoric_cols, cardinal_col, numeric_cols,

# Example usage with a specific column list for columns with missing values

stringtrainFeature_categoric_cols, cardinaltrainFeature_col, numerictrainFeature_cols = grab_col_names(trainFeaturescopy, 10, 20, betweenPercentValueColumnlistTrainFeatures)

stringtestFeature_categoric_cols, cardinaltestFeature_col, numerictestFeature_cols = grab_col_names(testUsers, 10, 20, betweenPercentValueColumnlistTestFeatures)

stringtrainUser_categoric_cols, cardinaltrainUser_col, numerictrainUser_cols = grab_col_names(trainUserscopy, 10, 20,betweenPercentValueColumnlistTrainUsers)
stringtestUser_categoric_cols, cardinaltestUser_col, numerictestUser_cols = grab_col_names(testUserscopy, 10, 20, betweenPercentValueColumnlistTestUsers)


def handle_outliers(dataframe, col_name, q1=0.25, q3=0.75, replace=True):
    # Aykırı değer sınırlarını hesapla
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range

    # Aykırı değer kontrolü yap
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        print(f"Aykırı değer bulundu: {col_name}")
        if replace:
            # Aykırı değerleri sınır değerlerle değiştir
            dataframe.loc[dataframe[col_name] > up_limit, col_name] = quartile3
            dataframe.loc[dataframe[col_name] < low_limit, col_name] = quartile1
            print(f"Aykırı değerler {col_name} sütununda sınırlarla değiştirildi.")
    else:
        print(f"Aykırı değer yok: {col_name}")

    return dataframe
liste=num_cols_testUsers+num_cols_testFeatures
liste.remove("ID")

for col in liste:
    train = handle_outliers(train, col)
    test = handle_outliers(test, col)



In [ ]:

# IAPRevenue için benzer sütunlar oluşturma
train["IAPRevenueTotal2"] = train["IAPRevenueD0"] + train["IAPRevenueD1"]
train["IAPRevenueTotal4"] = train["IAPRevenueD2"] + train["IAPRevenueD3"]
train["IAPRevenueTotal6"] = train["IAPRevenueD4"] + train["IAPRevenueD5"]
train["IAPRevenueTotal8"] = train["IAPRevenueD6"] + train["IAPRevenueD7"]
train["IAPRevenueTotal10"] = train["IAPRevenueD8"] + train["IAPRevenueD9"]
train["IAPRevenueTotal12"] = train["IAPRevenueD10"] + train["IAPRevenueD11"]
train["IAPRevenueTotal14"] = train["IAPRevenueD12"] + train["IAPRevenueD13"]
train["IAPRevenueTotal16"] = train["IAPRevenueD14"] + train["IAPRevenueD15"]


# LevelAdvancedCountTotal
train["LevelAdvancedCountTotal2"] = train["LevelAdvancedCountD0"] + train["LevelAdvancedCountD1"]
train["LevelAdvancedCountTotal4"] = train["LevelAdvancedCountD2"] + train["LevelAdvancedCountD3"]
train["LevelAdvancedCountTotal6"] = train["LevelAdvancedCountD4"] + train["LevelAdvancedCountD5"]
train["LevelAdvancedCountTotal8"] = train["LevelAdvancedCountD6"] + train["LevelAdvancedCountD7"]
train["LevelAdvancedCountTotal10"] = train["LevelAdvancedCountD8"] + train["LevelAdvancedCountD9"]
train["LevelAdvancedCountTotal12"] = train["LevelAdvancedCountD10"] + train["LevelAdvancedCountD11"]
train["LevelAdvancedCountTotal14"] = train["LevelAdvancedCountD12"] + train["LevelAdvancedCountD13"]
train["LevelAdvancedCountTotal16"] = train["LevelAdvancedCountD14"] + train["LevelAdvancedCountD15"]

# Level Duration
train["LevelDurationTotal2"] = train["Level_1_Duration"] + train["Level_2_Duration"]
train["LevelDurationTotal4"] = train["Level_3_Duration"] + train["Level_4_Duration"]
train["LevelDurationTotal6"] = train["Level_5_Duration"] + train["Level_6_Duration"]
train["LevelDurationTotal8"] = train["Level_7_Duration"] + train["Level_8_Duration"]
train["LevelDurationTotal10"] = train["Level_9_Duration"] + train["Level_10_Duration"]

# AdRevenue
train["AdRevenueTotal2"] = train["AdRevenueD0"] + train["AdRevenueD1"]
train["AdRevenueTotal4"] = train["AdRevenueD2"] + train["AdRevenueD3"]
train["AdRevenueTotal6"] = train["AdRevenueD4"] + train["AdRevenueD5"]
train["AdRevenueTotal8"] = train["AdRevenueD6"] + train["AdRevenueD7"]
train["AdRevenueTotal10"] = train["AdRevenueD8"] + train["AdRevenueD9"]
train["AdRevenueTotal12"] = train["AdRevenueD10"] + train["AdRevenueD11"]
train["AdRevenueTotal14"] = train["AdRevenueD12"] + train["AdRevenueD13"]
train["AdRevenueTotal16"] = train["AdRevenueD14"] + train["AdRevenueD15"]


# Retention için benzer sütunlar oluşturma (train seti)
train["RetentionTotal2"] = train["RetentionD0"] + train["RetentionD1"]
train["RetentionTotal4"] = train["RetentionD2"] + train["RetentionD3"]
train["RetentionTotal6"] = train["RetentionD4"] + train["RetentionD5"]
train["RetentionTotal8"] = train["RetentionD6"] + train["RetentionD7"]
train["RetentionTotal10"] = train["RetentionD8"] + train["RetentionD9"]
train["RetentionTotal12"] = train["RetentionD10"] + train["RetentionD11"]
train["RetentionTotal14"] = train["RetentionD12"] + train["RetentionD13"]
train["RetentionTotal16"] = train["RetentionD14"] + train["RetentionD15"]



# LevelAdvancedCountTotal
test["LevelAdvancedCountTotal2"] = test["LevelAdvancedCountD0"] + test["LevelAdvancedCountD1"]
test["LevelAdvancedCountTotal4"] = test["LevelAdvancedCountD2"] + test["LevelAdvancedCountD3"]
test["LevelAdvancedCountTotal6"] = test["LevelAdvancedCountD4"] + test["LevelAdvancedCountD5"]
test["LevelAdvancedCountTotal8"] = test["LevelAdvancedCountD6"] + test["LevelAdvancedCountD7"]
test["LevelAdvancedCountTotal10"] = test["LevelAdvancedCountD8"] + test["LevelAdvancedCountD9"]
test["LevelAdvancedCountTotal12"] = test["LevelAdvancedCountD10"] + test["LevelAdvancedCountD11"]
test["LevelAdvancedCountTotal14"] = test["LevelAdvancedCountD12"] + test["LevelAdvancedCountD13"]
test["LevelAdvancedCountTotal16"] = test["LevelAdvancedCountD14"] + test["LevelAdvancedCountD15"]

# Level Duration
test["LevelDurationTotal2"] = test["Level_1_Duration"] + test["Level_2_Duration"]
test["LevelDurationTotal4"] = test["Level_3_Duration"] + test["Level_4_Duration"]
test["LevelDurationTotal6"] = test["Level_5_Duration"] + test["Level_6_Duration"]
test["LevelDurationTotal8"] = test["Level_7_Duration"] + test["Level_8_Duration"]
test["LevelDurationTotal10"] = test["Level_9_Duration"] + test["Level_10_Duration"]

# AdRevenue
test["AdRevenueTotal2"] = test["AdRevenueD0"] + test["AdRevenueD1"]
test["AdRevenueTotal4"] = test["AdRevenueD2"] + test["AdRevenueD3"]
test["AdRevenueTotal6"] = test["AdRevenueD4"] + test["AdRevenueD5"]
test["AdRevenueTotal8"] = test["AdRevenueD6"] + test["AdRevenueD7"]
test["AdRevenueTotal10"] = test["AdRevenueD8"] + test["AdRevenueD9"]
test["AdRevenueTotal12"] = test["AdRevenueD10"] + test["AdRevenueD11"]
test["AdRevenueTotal14"] = test["AdRevenueD12"] + test["AdRevenueD13"]
test["AdRevenueTotal16"] = test["AdRevenueD14"] + test["AdRevenueD15"]


# IAPRevenue için benzer sütunlar oluşturma (test seti)
test["IAPRevenueTotal2"] = test["IAPRevenueD0"] + test["IAPRevenueD1"]
test["IAPRevenueTotal4"] = test["IAPRevenueD2"] + test["IAPRevenueD3"]
test["IAPRevenueTotal6"] = test["IAPRevenueD4"] + test["IAPRevenueD5"]
test["IAPRevenueTotal8"] = test["IAPRevenueD6"] + test["IAPRevenueD7"]
test["IAPRevenueTotal10"] = test["IAPRevenueD8"] + test["IAPRevenueD9"]
test["IAPRevenueTotal12"] = test["IAPRevenueD10"] + test["IAPRevenueD11"]
test["IAPRevenueTotal14"] = test["IAPRevenueD12"] + test["IAPRevenueD13"]
test["IAPRevenueTotal16"] = test["IAPRevenueD14"] + test["IAPRevenueD15"]


# Retention için benzer sütunlar oluşturma (test seti)
test["RetentionTotal2"] = test["RetentionD0"] + test["RetentionD1"]
test["RetentionTotal4"] = test["RetentionD2"] + test["RetentionD3"]
test["RetentionTotal6"] = test["RetentionD4"] + test["RetentionD5"]
test["RetentionTotal8"] = test["RetentionD6"] + test["RetentionD7"]
test["RetentionTotal10"] = test["RetentionD8"] + test["RetentionD9"]
test["RetentionTotal12"] = test["RetentionD10"] + test["RetentionD11"]
test["RetentionTotal14"] = test["RetentionD12"] + test["RetentionD13"]
test["RetentionTotal16"] = test["RetentionD14"] + test["RetentionD15"]



In [35]:

#LevelAdvancedCountTotal
train["LevelAdvancedCountTotal5"] = (
        train["LevelAdvancedCountD0"] + train["LevelAdvancedCountD1"] + train["LevelAdvancedCountD2"] + train["LevelAdvancedCountD3"]+ train["LevelAdvancedCountD4"]
)

train["LevelAdvancedCountTotal10"] = (
        train["LevelAdvancedCountD5"] + train["LevelAdvancedCountD6"] + train["LevelAdvancedCountD7"] + train["LevelAdvancedCountD8"]+ train["LevelAdvancedCountD9"]
)

train["LevelAdvancedCountTotal15"] = (
        train["LevelAdvancedCountD10"] + train["LevelAdvancedCountD11"] + train["LevelAdvancedCountD12"] + train["LevelAdvancedCountD13"]+ train["LevelAdvancedCountD14"]+train["LevelAdvancedCountD15"]
)

#Level Duration 
train["LevelDurationTotal5"] = (
        train["Level_1_Duration"] + train["Level_2_Duration"] + train["Level_3_Duration"] +
        train["Level_4_Duration"] + train["Level_5_Duration"]
)
train["LevelDurationTotal10"] = (
        train["Level_6_Duration"] + train["Level_7_Duration"] + train["Level_8_Duration"] +
        train["Level_9_Duration"] + train["Level_10_Duration"]
)





#AdRevenue
train["AdRevenueTotal5"] = (
        train["AdRevenueD0"] + train["AdRevenueD1"] + train["AdRevenueD2"] +
        train["AdRevenueD3"] + train["AdRevenueD4"]
)
train["AdRevenueTotal10"] = (
        train["AdRevenueD5"] + train["AdRevenueD6"] + train["AdRevenueD7"] +
        train["AdRevenueD8"] + train["AdRevenueD9"]
)
train["AdRevenueTotal15"] = (
        train["AdRevenueD10"] + train["AdRevenueD11"] + train["AdRevenueD12"] +
        train["AdRevenueD13"] + train["AdRevenueD14"]
)


# Retention Total
train["RetentionTotal5"] = (
        train["RetentionD0"] + train["RetentionD1"] + train["RetentionD2"] +
        train["RetentionD3"] + train["RetentionD4"]
)
train["RetentionTotal10"] = (
        train["RetentionD5"] + train["RetentionD6"] + train["RetentionD7"] +
        train["RetentionD8"] + train["RetentionD9"]
)
train["RetentionTotal15"] = (
        train["RetentionD10"] + train["RetentionD11"] + train["RetentionD12"] +
        train["RetentionD13"] + train["RetentionD14"]
)

# IAPRevenue Total
train["IAPRevenueTotal5"] = (
        train["IAPRevenueD0"] + train["IAPRevenueD1"] + train["IAPRevenueD2"] +
        train["IAPRevenueD3"] + train["IAPRevenueD4"]
)
train["IAPRevenueTotal10"] = (
        train["IAPRevenueD5"] + train["IAPRevenueD6"] + train["IAPRevenueD7"] +
        train["IAPRevenueD8"] + train["IAPRevenueD9"]
)
train["IAPRevenueTotal15"] = (
        train["IAPRevenueD10"] + train["IAPRevenueD11"] + train["IAPRevenueD12"] +
        train["IAPRevenueD13"] + train["IAPRevenueD14"]
)











#test






#LevelAdvancedCountTotal
test["LevelAdvancedCountTotal5"] = (
        test["LevelAdvancedCountD0"] + test["LevelAdvancedCountD1"] + test["LevelAdvancedCountD2"] + test["LevelAdvancedCountD3"]+ test["LevelAdvancedCountD4"]
)

test["LevelAdvancedCountTotal10"] = (
        test["LevelAdvancedCountD5"] + test["LevelAdvancedCountD6"] + test["LevelAdvancedCountD7"] + test["LevelAdvancedCountD8"]+ test["LevelAdvancedCountD9"]
)

test["LevelAdvancedCountTotal15"] = (
        test["LevelAdvancedCountD10"] + test["LevelAdvancedCountD11"] + test["LevelAdvancedCountD12"] + test["LevelAdvancedCountD13"]+ test["LevelAdvancedCountD14"]+test["LevelAdvancedCountD15"]
)


#Level Duration 
test["LevelDurationTotal5"] = (
        test["Level_1_Duration"] + test["Level_2_Duration"] + test["Level_3_Duration"] +
        test["Level_4_Duration"] + test["Level_5_Duration"]
)
test["LevelDurationTotal10"] = (
        test["Level_6_Duration"] + test["Level_7_Duration"] + test["Level_8_Duration"] +
        test["Level_9_Duration"] + test["Level_10_Duration"]
)





#AdRevenue
test["AdRevenueTotal5"] = (
        test["AdRevenueD0"] + test["AdRevenueD1"] + test["AdRevenueD2"] +
        test["AdRevenueD3"] + test["AdRevenueD4"]
)
test["AdRevenueTotal10"] = (
        test["AdRevenueD5"] + test["AdRevenueD6"] + test["AdRevenueD7"] +
        test["AdRevenueD8"] + test["AdRevenueD9"]
)
test["AdRevenueTotal15"] = (
        test["AdRevenueD10"] + test["AdRevenueD11"] + test["AdRevenueD12"] +
        test["AdRevenueD13"] + test["AdRevenueD14"]
)

# Aynı işlemleri test setine de uyguluyoruz
test["RetentionTotal5"] = (
        test["RetentionD0"] + test["RetentionD1"] + test["RetentionD2"] +
        test["RetentionD3"] + test["RetentionD4"]
)
test["RetentionTotal10"] = (
        test["RetentionD5"] + test["RetentionD6"] + test["RetentionD7"] +
        test["RetentionD8"] + test["RetentionD9"]
)
test["RetentionTotal15"] = (
        test["RetentionD10"] + test["RetentionD11"] + test["RetentionD12"] +
        test["RetentionD13"] + test["RetentionD14"]
)


# Aynı işlemleri test setine de uyguluyoruz
test["IAPRevenueTotal5"] = (
        test["IAPRevenueD0"] + test["IAPRevenueD1"] + test["IAPRevenueD2"] +
        test["IAPRevenueD3"] + test["IAPRevenueD4"]
)
test["IAPRevenueTotal10"] = (
        test["IAPRevenueD5"] + test["IAPRevenueD6"] + test["IAPRevenueD7"] +
        test["IAPRevenueD8"] + test["IAPRevenueD9"]
)
test["IAPRevenueTotal15"] = (
        test["IAPRevenueD10"] + test["IAPRevenueD11"] + test["IAPRevenueD12"] +
        test["IAPRevenueD13"] + test["IAPRevenueD14"]
)





In [36]:






train["first_prediction"]=train["first_prediction"].interpolate(method="linear")
test["first_prediction"]=test["first_prediction"].interpolate(method="linear")





#BAŞARISIZ
#network_revenue_avg = test.groupby('ad_network')['Total_Revenue'].mean().reset_index()
#network_revenue_avg.columns = ['ad_network', 'AdNetwork_Revenue_Avg']

# Kullanıcı verisine ekleyelim
#test = test.merge(network_revenue_avg, on='ad_network', how='left')
#train["first_open_timestamp"]=np.log1p(train["first_open_timestamp"])
#test["first_open_timestamp"]=np.log1p(test["first_open_timestamp"])

#test["local_first_open_timestamp"]=np.log1p(test["local_first_open_timestamp"])
#train["local_first_open_timestamp"]=np.log1p(train["local_first_open_timestamp"])


# İlk IAP satın almasının yapıldığı günü bulmak için bir fonksiyon yazalım
#def days_to_first_iap(row):
#    for i in range(1, 16):
#        if row[f'IAPRevenueD{i}'] > 0:
#            return i
#    return None

# Kullanıcı başına kaçıncı gün ilk satın almanın yapıldığını hesaplayalım
#train['Days_to_First_IAP'] = train.apply(days_to_first_iap, axis=1)
#test["Days_to_First_IAP"]=train.apply(days_to_first_iap,axis=1)




####################
train['Total_Revenue'] = train.filter(like='AdRevenueD').sum(axis=1) + train.filter(like='IAPRevenueD').sum(axis=1)
test['Total_Revenue'] = test.filter(like='AdRevenueD').sum(axis=1) + test.filter(like='IAPRevenueD').sum(axis=1)
######################

#############################
network_revenue_avg = train.groupby('ad_network')['Total_Revenue'].mean().reset_index()
network_revenue_avg.columns = ['ad_network', 'AdNetwork_Revenue_Avg']
train = train.merge(network_revenue_avg, on='ad_network', how='left')


network_revenue_avg = test.groupby('ad_network')['Total_Revenue'].mean().reset_index()
network_revenue_avg.columns = ['ad_network', 'AdNetwork_Revenue_Avg']
test = test.merge(network_revenue_avg, on='ad_network', how='left')
###############################





################################
# Device Brand'e göre ortalama gelir
device_brand_revenue_avg = train.groupby('device_brand')['Total_Revenue'].mean().reset_index()
device_brand_revenue_avg.columns = ['device_brand', 'Device_Brand_Revenue_Avg']
train = train.merge(device_brand_revenue_avg, on='device_brand', how='left')

device_brand_revenue_avg = test.groupby('device_brand')['Total_Revenue'].mean().reset_index()
device_brand_revenue_avg.columns = ['device_brand', 'Device_Brand_Revenue_Avg']
test = test.merge(device_brand_revenue_avg, on='device_brand', how='left')
###############################


#############################
# Ülke, cihaz ve platform kombinasyonuna göre ortalama gelir
country_device_platform_revenue_avg = train.groupby(['country', 'device_category', 'platform'])['Total_Revenue'].mean().reset_index()
country_device_platform_revenue_avg.columns = ['country', 'device_category', 'platform', 'Country_Device_Platform_Revenue_Avg']
train = train.merge(country_device_platform_revenue_avg, on=['country', 'device_category', 'platform'], how='left')


country_device_platform_revenue_avg = test.groupby(['country', 'device_category', 'platform'])['Total_Revenue'].mean().reset_index()
country_device_platform_revenue_avg.columns = ['country', 'device_category', 'platform', 'Country_Device_Platform_Revenue_Avg']
test = test.merge(country_device_platform_revenue_avg, on=['country', 'device_category', 'platform'], how='left')
###############################

#train["first_open_date"] = pd.to_datetime(train["first_open_date"])
#train["first_open_timestamp"] = pd.to_timedelta(train["first_open_timestamp"])
#train["local_first_open_timestamp"] = pd.to_timedelta(train["local_first_open_timestamp"])

#test["first_open_date"] = pd.to_datetime(test["first_open_date"])
#test["first_open_timestamp"] = pd.to_timedelta(test["first_open_timestamp"])
#test["local_first_open_timestamp"] = pd.to_timedelta(test["local_first_open_timestamp"])



In [37]:
#score 1 BAŞARILI
train["score1"]=(train["LevelAdvancedCountD10"]+train["LevelAdvancedCountD11"]+train["LevelAdvancedCountD12"]+train["LevelAdvancedCountD13"])*train["Total_Revenue"]
test["score1"]=(test["LevelAdvancedCountD10"]+test["LevelAdvancedCountD11"]+test["LevelAdvancedCountD12"]+test["LevelAdvancedCountD13"])*test["Total_Revenue"]

#score 2 BAŞARILI
train["score2"]=(train["Country_Device_Platform_Revenue_Avg"]+train["first_prediction"])**2
test["score2"]=(test["Country_Device_Platform_Revenue_Avg"]+test["first_prediction"])**2

#Score3 BAŞARISIZ
#train["score3"]=(train["AdRevenueD13"]+train["AdRevenueD1"]+train["AdRevenueD12"]+train["AdRevenueD3"]+train["AdRevenueD4"])**2

#test["score3"]=(test["AdRevenueD13"]+test["AdRevenueD1"]+test["AdRevenueD12"]+test["AdRevenueD3"]+test["AdRevenueD4"])**2

#Score 3 BAŞARILI
train["score3"]=(train["LevelAdvancedCountTotal15"]+train["LevelAdvancedCountTotal10"]+train["LevelAdvancedCountTotal5"])**2
test["score3"]=(test["LevelAdvancedCountTotal15"]+test["LevelAdvancedCountTotal10"]+train["LevelAdvancedCountTotal5"])**2



In [38]:
train["score4"]=(train["Total_Revenue"]+train["first_prediction"])**2
test["score4"]=(test["Total_Revenue"]+test["first_prediction"])**2


In [39]:
def outlier_thresholds(dataframe, col_name, q1=0.25, q3=0.75):
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def check_outlier(dataframe, col_name):
    low_limit, up_limit = outlier_thresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

liste=["AdRevenueD0","AdRevenueD1"]

for col in liste:
    print(col, outlier_thresholds(train, col))
    print(col, outlier_thresholds(test, col))

for col in liste:
    print(col, check_outlier(train, col))
    print(col, check_outlier(test, col))

for col in liste:
    col, replace_with_thresholds(train, col)
    col, replace_with_thresholds(test, col)
    






AdRevenueD0 (-0.059846108900488616, 0.0997435148341477)
AdRevenueD0 (-0.05986737250494386, 0.09977895417490643)
AdRevenueD1 (-0.008947809449015288, 0.014913015748358814)
AdRevenueD1 (-0.009000000078230999, 0.015000000130384999)
AdRevenueD0 True
AdRevenueD0 True
AdRevenueD1 True
AdRevenueD1 True


In [40]:
#silinecekler=pd.read_csv("SİLİNECEKLER.csv")
#silinecek=silinecekler["Feature"]

#a=test.columns
# Ortak elemanları bulmak için set kesişimini kullanıyoruz
#ortak_liste = list(set(silinecek) & set(a))


#train.drop(columns=ortak_liste,axis=1,inplace=True)
#test.drop(columns=ortak_liste,axis=1,inplace=True)
#train.shape,test.shape

In [41]:
train["first_prediction"]=train["first_prediction"].interpolate(method="linear")
test["first_prediction"]=test["first_prediction"].interpolate(method="linear")

train['ad_network'] = train['ad_network'].fillna('organic')
test['ad_network'] = test['ad_network'].fillna('organic')


# OrdinalEncoder'ı tanıml
oe = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)

# Tüm string kolonları bul
string_cols = test.select_dtypes(include=['object',"bool"]).columns

# Tüm string kolonları dolas
for col in string_cols:
    # Convert column to string type to handle any mixed data types
    test[col] = train[col].astype(str)
    train[col] = test[col].astype(str)

    # Fit OrdinalEncoder
    train[[col]] = oe.fit_transform(train[[col]])
    test[[col]] = oe.transform(test[[col]])
train.shape,test.shape

((878594, 110), (585730, 109))

In [42]:
num_cols_testFeatures


['ID',
 'first_prediction',
 'LevelAdvancedCountD0',
 'LevelAdvancedCountD1',
 'LevelAdvancedCountD2',
 'LevelAdvancedCountD3',
 'LevelAdvancedCountD4',
 'LevelAdvancedCountD5',
 'LevelAdvancedCountD6',
 'LevelAdvancedCountD7',
 'LevelAdvancedCountD8',
 'LevelAdvancedCountD9',
 'LevelAdvancedCountD10',
 'LevelAdvancedCountD11',
 'LevelAdvancedCountD12',
 'LevelAdvancedCountD13',
 'LevelAdvancedCountD14',
 'LevelAdvancedCountD15',
 'Level_1_Duration',
 'Level_2_Duration',
 'Level_3_Duration',
 'Level_4_Duration',
 'Level_5_Duration',
 'Level_6_Duration',
 'Level_7_Duration',
 'Level_8_Duration',
 'Level_9_Duration',
 'Level_10_Duration',
 'AdRevenueD0',
 'AdRevenueD1',
 'AdRevenueD2',
 'AdRevenueD3',
 'AdRevenueD4',
 'AdRevenueD5',
 'AdRevenueD6',
 'AdRevenueD7',
 'AdRevenueD8',
 'AdRevenueD9',
 'AdRevenueD10',
 'AdRevenueD11',
 'AdRevenueD12',
 'AdRevenueD13',
 'AdRevenueD14',
 'AdRevenueD15']

In [43]:
print(train.isnull().sum().sum(),test.isnull().sum().sum())
train.fillna(-1,axis=1,inplace=True),test.fillna(-1,axis=1,inplace=True)
print(train.isnull().sum().sum(),test.isnull().sum().sum())


10441335 1884072
0 0


In [44]:
y = train["TARGET"]
X = train.drop(columns=["TARGET"],axis=1)



# Repeated K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=24)
scores = []
test_preds = np.zeros(len(test))  # Assuming `test` is your test set

# Perform K-Fold Cross-Validation
for fold, (train_index, test_index) in enumerate(kf.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Create and train CatBoostRegressor model
    model = CatBoostRegressor(random_state=41,verbose=False)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate RMSE score
    score = mean_squared_error(y_test, y_pred,squared=False)  # RMSE
    scores.append(score)

    print(f"Fold {fold + 1}: RMSE = {score:.4f}")

    # Average test set predictions across folds
    test_preds += model.predict(test) / kf.n_splits

# Calculate average RMSE score
mean_score = np.mean(scores)
print(f"Average RMSE = {mean_score:.4f}")

Fold 1: RMSE = 1.8783
Fold 2: RMSE = 1.3093
Fold 3: RMSE = 1.6310
Fold 4: RMSE = 1.5847
Fold 5: RMSE = 1.6869
Average RMSE = 1.6180


In [52]:
data=pd.DataFrame({"ID":ID,"TARGET":test_preds})
data.set_index("ID",inplace=True)
data.to_csv("submission7.csv")

In [53]:
data.head()
#1.6020 num_but_cat columnlar
#1.6047 cat_but_not_car_testUsers+cat_but_not_car_testFeatures
#1.5963 cat_but_car_testFeatures+cat_but_car_testUsers
#1.6047   cat_cols_testFeatures+cat_cols_testUsers
#1.5963          outlier değer olmadan

#1.5959

#1.6020 num_but_cat columnlar
#1.6047 cat_but_not_car_testUsers+cat_but_not_car_testFeatures
#1.5963 cat_but_car_testFeatures+cat_but_car_testUsers
#1.6047  cat_cols_testFeatures+cat_cols_testUsers
# 2.5      num_cols_testUsers+num_cols_testFeatures
#1.65
#1.6497
#1.6095
#1.6130 => en iyi skor

,TARGET
ID,
878594,0.014308
878595,0.009347
878596,0.017519
878597,0.563819
878598,0.017639


# OrdinalEncoder'ı tanımla
oe = OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
traincopy=train[:10000]
a=traincopy["TARGET"]




# Tüm string kolonları bul
string_cols = test.select_dtypes(include=['object',"bool"]).columns

# Tüm string kolonları dolas
for col in string_cols:
    # Convert column to string type to handle any mixed data types
    traincopy[col] = traincopy[col].astype(str)

    # Fit OrdinalEncoder
    traincopy[[col]] = oe.fit_transform(traincopy[[col]])
    
traincopy=traincopy[trainFeaturescopy.columns]

traincopy=pd.concat([traincopy,a],axis=1)
traincopy.drop("ID",axis=1)

#RetentionTotal
traincopy["RetentionTotal5"] = (
        traincopy["RetentionD0"] + traincopy["RetentionD1"] + traincopy["RetentionD2"] +
        traincopy["RetentionD3"] + traincopy["RetentionD4"]
)
traincopy["RetentionTotal10"] = (
        traincopy["RetentionD5"] + traincopy["RetentionD6"] + traincopy["RetentionD7"] +
        traincopy["RetentionD8"] + traincopy["RetentionD9"]
)
traincopy["RetentionTotal15"] = (
        traincopy["RetentionD10"] + traincopy["RetentionD11"] + traincopy["RetentionD12"] +
        traincopy["RetentionD13"] + traincopy["RetentionD14"] + traincopy["RetentionD15"]
)

#LevelAdvancedCountTotal
traincopy["LevelAdvancedCountTotal5"] = (
        traincopy["LevelAdvancedCountD0"] + traincopy["LevelAdvancedCountD1"] + traincopy["LevelAdvancedCountD2"] + traincopy["LevelAdvancedCountD3"]+ traincopy["LevelAdvancedCountD4"]
)

traincopy["LevelAdvancedCountTotal10"] = (
        traincopy["LevelAdvancedCountD5"] + traincopy["LevelAdvancedCountD6"] + traincopy["LevelAdvancedCountD7"] + traincopy["LevelAdvancedCountD8"]+ traincopy["LevelAdvancedCountD9"]
)

traincopy["LevelAdvancedCountTotal15"] = (
        traincopy["LevelAdvancedCountD10"] + traincopy["LevelAdvancedCountD11"] + traincopy["LevelAdvancedCountD12"] + traincopy["LevelAdvancedCountD13"]+ traincopy["LevelAdvancedCountD14"]+traincopy["LevelAdvancedCountD15"]
)

liste=["LevelDurationTotal5","LevelDurationTotal10","LevelAdvancedCountTotal5","LevelAdvancedCountTotal10","LevelAdvancedCountTotal15",
       "RetentionTotal5","RetentionTotal10","RetentionTotal15","IAPRevenueTotal5","IAPRevenueTotal15"]
#Level Duration 
traincopy["LevelDurationTotal5"] = (
        traincopy["Level_1_Duration"] + traincopy["Level_2_Duration"] + traincopy["Level_3_Duration"] +
        traincopy["Level_4_Duration"] + traincopy["Level_5_Duration"]
)
traincopy["LevelDurationTotal10"] = (
        traincopy["Level_6_Duration"] + traincopy["Level_7_Duration"] + traincopy["Level_8_Duration"] +
        traincopy["Level_9_Duration"] + traincopy["Level_10_Duration"]
)





#AdRevenue
traincopy["AdRevenueTotal5"] = (
        traincopy["AdRevenueD0"] + traincopy["AdRevenueD1"] + traincopy["AdRevenueD2"] +
        traincopy["AdRevenueD3"] + traincopy["AdRevenueD4"]
)
traincopy["AdRevenueTotal10"] = (
        traincopy["AdRevenueD5"] + traincopy["AdRevenueD6"] + traincopy["AdRevenueD7"] +
        traincopy["AdRevenueD8"] + traincopy["AdRevenueD9"]
)
traincopy["AdRevenueTotal15"] = (
        traincopy["AdRevenueD10"] + traincopy["AdRevenueD11"] + traincopy["AdRevenueD12"] +
        traincopy["AdRevenueD13"] + traincopy["AdRevenueD14"]
)




traincopy["IAPRevenueTotal5"] = (
        traincopy["IAPRevenueD0"] + traincopy["IAPRevenueD1"] + traincopy["IAPRevenueD2"] +
        traincopy["IAPRevenueD3"] + traincopy["IAPRevenueD4"]
)
traincopy["IAPRevenueTotal10"] = (
        traincopy["IAPRevenueD5"] + traincopy["IAPRevenueD6"] + traincopy["IAPRevenueD7"] +
        traincopy["IAPRevenueD8"] + traincopy["IAPRevenueD9"]
)
traincopy["IAPRevenueTotal15"] = (
        traincopy["IAPRevenueD10"] + traincopy["IAPRevenueD11"] + traincopy["IAPRevenueD12"] +
        traincopy["IAPRevenueD13"] + traincopy["IAPRevenueD14"]
)
traincopy


#TEST ET NÜMERİK
from sklearn.impute import IterativeImputer


def fill_na_forward(data, col):
    data[col] = train[col]
    print(data[col].isnull().sum())
    data[col].fillna(method='ffill', inplace=True)

def fill_na_backward(data, col):
    data[col] = train[col]
    print(data[col].isnull().sum())
    data[col].fillna(method='bfill', inplace=True)

def fill_na_mean(data, col):
    data[col] = train[col]
    print(data[col].isnull().sum())
    mean_value = data[col].mean()
    data[col].fillna(mean_value, inplace=True)

def fill_na_median(data, col):
    data[col] = train[col]
    print(data[col].isnull().sum())
    median_value = data[col].median()
    data[col].fillna(median_value, inplace=True)

def interpolate_values(data, col):
    data[col] = train[col]
    print(data[col].isnull().sum())
    data[col].interpolate(method='linear', inplace=True)

def RandomForestImpute(data, target_cols):
    imputer = IterativeImputer(estimator=RandomForestRegressor())
    data[target_cols] = imputer.fit_transform(data[target_cols])

imputation_methods = [
    fill_na_forward,
    fill_na_backward,
    fill_na_mean,
    fill_na_median,
    interpolate_values,
]

def apply_ml_and_report(data, target, method_name):
    """
    This function sets up the AutoML pipeline using the imputed data and trains
    the model to get the performance score.
    """
    # Run AutoML using pycaret
    clf = setup(data, target=target, html=False)
    best_model = compare_models()

    # Print or log the result along with the imputation method
    print(f"Age column imputed with {method_name}, best model: {best_model}")

# Loop through each method and apply it to the VIP column, then run AutoML on the Transported column
for method in imputation_methods:
    # Copy the data for each iteration to keep the imputations isolated
    data_copy = traincopy.copy()

    # Name of the current method for logging
    method_name = method.__name__

    # Apply the imputation method
    target_col = 'first_prediction'
    
    if method_name == "RandomForestImpute":
        target_features = liste
        RandomForestImpute(data_copy, target_features)  # Pass target_features as an argument
    else:
        method(data_copy, target_col)  # Apply other imputation methods

    # Now apply AutoML to the "Transported" column
    apply_ml_and_report(data_copy, "first_prediction", method_name)

#silme

#prediction Trainuser randomforest

def fill_na_median(data, col):  
    median_value = data[col].median()
    data[col].fillna(median_value, inplace=True)


def RandomForestImpute(data, targetFeatureList, target):
    # Hedef sütunun targetFeatureList içinde olmamasını sağla
    targetFeatureList = [col for col in targetFeatureList if col in data.columns and col != target]

    # Eksik verileri geçici olarak -1 ile doldur
    data[targetFeatureList] = data[targetFeatureList].fillna(-1)


    # Eğitim ve test setlerini oluştur
    # Sadece hedef sütundaki eksik ve eksiksiz veriler üzerinde işlem yap
    train_data_new = data[data[target].notnull()]
    test_data_new = data[data[target].isnull()]

    # Hedef ve özellik sütunlarını ayır
    X_train = train_data_new[targetFeatureList]
    y_train = train_data_new[target]
    X_test = test_data_new[targetFeatureList]



    # Modeli eğit ve eksik verileri tahmin et
    model = RandomForestRegressor(random_state=42, n_estimators=100)
    model.fit(X_train, y_train)

    # Eksik olan değerleri tahmin et
    y_pred = model.predict(X_test)

    # Tahmin edilen değerleri test setine ekle
    data.loc[data[target].isnull(), target] = y_pred

    # Eksik olan -1 değerlerini NaN ile değiştir
    liste = ["RoomService", "ShoppingMall", "Spa", "VRDeck", "Age","FoodCourt"]
    for i in liste:
        if i != target:
            data[i] = data[i].replace(-1, np.nan)
    
    return data
# RandomForestImpute fonksiyonunu uygulayın


RandomForestImpute(train, ['Age', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'RoomService'], 'RoomService')
RandomForestImpute(test, ['Age', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'RoomService'], 'RoomService')
RandomForestImpute(train, ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Age'], 'Age')
RandomForestImpute(test, ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Age'], 'Age')
RandomForestImpute(train, ['Age', 'FoodCourt', 'Spa', 'VRDeck', 'RoomService', 'ShoppingMall'], 'ShoppingMall')
RandomForestImpute(test, ['Age', 'FoodCourt', 'Spa', 'VRDeck', 'RoomService', 'ShoppingMall'], 'ShoppingMall')
RandomForestImpute(train, ['Age', 'FoodCourt', 'ShoppingMall', 'VRDeck', 'RoomService', 'Spa'], 'Spa')
RandomForestImpute(test, ['Age', 'FoodCourt', 'ShoppingMall', 'VRDeck', 'RoomService', 'Spa'], 'Spa')
RandomForestImpute(train, ['Age', 'FoodCourt', 'ShoppingMall', 'Spa', 'RoomService', 'VRDeck'], 'VRDeck')
RandomForestImpute(test, ['Age', 'FoodCourt', 'ShoppingMall', 'Spa', 'RoomService', 'VRDeck'], 'VRDeck')
RandomForestImpute(train, ['Age', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'value2'], 'value2')
RandomForestImpute(test, ['Age', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'value2'], 'value2')
fill_na_median(train, 'FoodCourt')
fill_na_median(test, 'FoodCourt')

In [47]:
# TEST ET CATEGORİK

# Ordinal Encoding for string columns
# oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# Find all string columns
# string_cols = test.select_dtypes(include=['object']).columns

# Loop over all string columns and encode them
# for col in string_cols:
#     if col=="VIP":
#         print("")
#     else:
#         train[col] = train[col].astype(str)
#         test[col] = test[col].astype(str)
        
#         Fit OrdinalEncoder
#         train[[col]] = oe.fit_transform(train[[col]])
#         test[[col]] = oe.transform(test[[col]])

# print(train["VIP"].isnull().sum())
# Copy the original data
# trainData = train.copy()
# testData = test.copy()
# Ordinal Encoding for string columns
# oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

# Find all string columns
# string_cols = test.select_dtypes(include=['object']).columns

# Loop over all string columns and encode them
# for col in string_cols:
#     if col=="VIP":
#         print("")
#     else:
#         train[col] = train[col].astype(str)
#         test[col] = test[col].astype(str)
        
#         Fit OrdinalEncoder
#         train[[col]] = oe.fit_transform(train[[col]])
#         test[[col]] = oe.transform(test[[col]])
# print(train["VIP"].isnull().sum())

# Imputation functions
# def fill_na_forward(data, col):
#     data[col]=train[col]
#     print(train["VIP"].isnull().sum())  
#     data[col] = data[col].ffill()

# def fill_na_backward(data, col):
#     data[col]=train[col]
#     print(train["VIP"].isnull().sum())   
#     data[col] = data[col].bfill()

# def fill_na_mode(data, col):
#     data[col]=train[col]
#     print(train["VIP"].isnull().sum())
#     mode_value = data[col].mode()[0]
#     data[col].fillna(mode_value, inplace=True)

# def interpolate_values(data, col):
#     data[col]=train[col]
#     print(train[col].isnull().sum())
#     data[col] = data[col].interpolate(method='linear')

# def knn_imputation(data, target_cols):
#     data[col]=train[col]
#     print(train["VIP"].isnull().sum())
#     """
#     Perform K-Nearest Neighbors Imputation on the target columns.
#     """
#     knn = KNNImputer()
#     data[target_cols] = knn.fit_transform(data[target_cols])

# List of imputation methods
# imputation_methods = [
#     fill_na_forward,
#     fill_na_backward,
#     fill_na_mode,
#     interpolate_values,
#     knn_imputation
# ]

# def apply_ml_and_report(data, target, method_name):
#     """
#     This function sets up the AutoML pipeline using the imputed data and trains
#     the model to get the performance score.
#     """
#     Run AutoML using pycaret
#     clf = setup(data, target=target, html=False,random_state=41)
#     best_model = compare_models()

#     Print or log the result along with the imputation method
#     print(f"VIP column imputed with {method_name}, best model: {best_model}")

# Loop through each method and apply it to the VIP column, then run AutoML on the Transported column
# for method in imputation_methods:
#     Copy the data for each iteration to keep the imputations isolated
#     data_copy = trainData.copy()

#     Name of the current method for logging
#     method_name = method.__name__

#     Try to apply the imputation method
#     try:
#         Impute the "VIP" column
#         Bu column yerine verilen columnu yazıcaksınız
#         target_col = 'VIP'
        
#         if method_name == "knn_imputation":
#             target_features = ['HomePlanet', 'CryoSleep', 'Destination', 'value1', 'value3', 'Transported']
#             knn_imputation(data_copy, target_features)  # Apply KNNImputer on selected columns
#         else:
#             method(data_copy, target_col)  # Apply other imputation methods
            

#         Now apply AutoML to the "Transported" column
#         apply_ml_and_report(data_copy, "Transported", method_name)

#     except Exception as e:
#         print(f"An error occurred with method {method_name}: {e}")

In [48]:
# Scaling the data
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)


train=train.iloc[:5000]
X_train = train.drop(columns=["TARGET"]) 
X_test = test                              
y_train = train["TARGET"]    


transformers.logging.get_verbosity = lambda: logging.NOTSET
transformers.logging.get_verbosity()

clf1=setup(train, target="TARGET",html=False)
best_model=compare_models()
#The best model is XGBBoost, for now :)


In [49]:
train.shape,test.shape

((878594, 110), (585730, 109))

import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


# Verinizi işleyin (datetime sütunlarını ayrıştırın)
#train = process_datetime_features(train)
#test = process_datetime_features(test)

# Hedef ve özellikleri ayırın
y = train["TARGET"]
X = train.drop(columns=["TARGET"], axis=1)


# Repeated K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=24)
scores = []
test_preds = np.zeros(len(test))  # Test setinizin boyutuna göre ayarlayın

# Model parametreleri
param = {
    'iterations': 456,
    'depth': 4,
    'learning_rate': 0.017250413325681634,
    'random_strength': 8,
    'bagging_temperature': 0.5608982636794247,
    'border_count': 208,
    'l2_leaf_reg': 9.268604139004962,
    'early_stopping_rounds': 50,  # Erken durdurma
    'verbose': 100
}

# K-Fold Cross-Validation
for fold, (train_index, test_index) in enumerate(kf.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]

    # CatBoost Regressor modelini oluştur
    model = CatBoostRegressor(**param, random_state=41)

    # Modeli eğit
    model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)

    # Tahminler yap
    y_pred = model.predict(X_val)

    # RMSE hesapla
    score = mean_squared_error(y_val, y_pred, squared=False)  # RMSE
    scores.append(score)

    print(f"Fold {fold + 1}: RMSE = {score:.4f}")

    # Test seti için ortalama tahminler al
    test_preds += model.predict(test) / kf.n_splits

# Ortalama RMSE
mean_score = np.mean(scores)
print(f"Ortalama RMSE = {mean_score:.4f}")




import shap
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
import numpy as np

# Özellik ölçeklendirme (isteğe bağlı)
scaler = StandardScaler()

y = train["TARGET"]
X = train.drop(columns=["TARGET"], axis=1)

param = {
    "iterations": 163,
    "depth": 5,
    "learning_rate": 0.4330894758291788,
    "random_strength": 9,
    "bagging_temperature": 0.07827553008007726,
    "border_count": 96,
    "l2_leaf_reg": 4.5286949510996,
}

# Repeated K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=24)
scores = []
test_preds = np.zeros(len(test))  # Assuming `test` is your test set

# SHAP değerlerini saklayacağımız liste
all_shap_values = []
X_all_train = []

# Perform K-Fold Cross-Validation
for fold, (train_index, test_index) in enumerate(kf.split(X, y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Create and train CatBoostRegressor model
    model = CatBoostRegressor(**param, random_state=41, verbose=False)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate RMSE score
    score = mean_squared_error(y_test, y_pred, squared=False)  # RMSE
    scores.append(score)

    print(f"Fold {fold + 1}: RMSE = {score:.4f}")

    # SHAP değerlerini her fold için hesapla
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_train)

    # SHAP değerlerini ve X_train'i bir listede sakla
    all_shap_values.append(shap_values)
    X_all_train.append(X_train)

    # Average test set predictions across folds
    test_preds += model.predict(test) / kf.n_splits

# Calculate average RMSE score
mean_score = np.mean(scores)
print(f"Average RMSE = {mean_score:.4f}")

# Tüm SHAP değerlerini ve X_train'leri birleştir
combined_shap_values = np.concatenate(all_shap_values, axis=0)
combined_X_train = pd.concat(X_all_train)

# SHAP değerlerini görselleştir
shap.summary_plot(combined_shap_values, combined_X_train)



shap.summary_plot(combined_shap_values, combined_X_train, max_display=50)


# SHAP değerlerinin ortalamasını (mutlak değerde) hesapla
shap_values_mean = np.mean(np.abs(combined_shap_values), axis=0)

# Özellik isimlerini al
feature_names = combined_X_train.columns

# SHAP değerlerinin ortalamasını ve isimleri bir DataFrame'e ekle
shap_summary = pd.DataFrame({
    'Feature': feature_names,
    'Mean_SHAP_Value': shap_values_mean
})

# SHAP değerlerine göre artan şekilde sıralama yap ve en önemsiz 5 özelliği al
least_important_features = shap_summary.sort_values(by='Mean_SHAP_Value').head(30)

# En önemsiz 5 özelliği göster
print(least_important_features)

least_important_features["SİLİNECEKFUTURE"]=least_important_features["Feature"]
least_important_features.to_csv("SİLİNECEKLER.csv")

shap.waterfall_plot(shap.Explanation(values=combined_shap_values[0], base_values=explainer.expected_value, data=combined_X_train.iloc[0]))



# # Prepare your data
y = train["TARGET"]
X = train.drop(columns=["TARGET"])
test_preds = np.zeros(len(test))  # Assuming `test` is your test set

def objective(trial):
     # Define the hyperparameters to tune
     param = {
         'iterations': trial.suggest_int('iterations', 100, 1000),
         'depth': trial.suggest_int('depth', 1, 16),
         'learning_rate': trial.suggest_float('learning_rate', 0.01, 1),
         'random_strength': trial.suggest_int('random_strength', 1, 20),
         'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
         'border_count': trial.suggest_int('border_count', 50, 255),
         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.01, 10.0),
         'early_stopping_rounds': trial.suggest_int('early_stopping_rounds',0,700),
         'random_state': 42
     }

     # Repeated K-Fold cross-validation
     kf = KFold(n_splits=5, shuffle=True, random_state=24)
     scores = []

     for train_index, test_index in kf.split(X, y):
         X_train, X_test = X.iloc[train_index], X.iloc[test_index]
         y_train, y_test = y.iloc[train_index], y.iloc[test_index]

         # Create and train CatBoostClassifier model with parameters
         model = CatBoostRegressor(**param, verbose=False)
         model.fit(X_train, y_train)

         # Predict and calculate accuracy
         y_pred = model.predict(X_test)
         score = mean_squared_error(y_test, y_pred)
         scores.append(score)

     # Calculate the mean of the scores
     mean_score = np.mean(scores)
     return mean_score

 # Create a study object and specify the direction of the optimization
study = optuna.create_study()
study.optimize(objective, n_trials=5)  # Specify the number of trials

# # Best hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# # You can retrain your model on the full training set using
